We find out the number of monomers from the center of the condensate with delta_R=1 sigma, and then divide it by the volume of the slice which is area of the slice* delta_R (=1). 

In [1]:
import numpy as np
import scipy as sp
import MDAnalysis as mdan
from MDAnalysis.analysis import rdf as mAnRDF
from MDAnalysis.analysis import distances as mAnDist
from MDAnalysis.analysis.base import AnalysisFromFunction as AF
from MDAnalysis.analysis import distances
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import measure

sns.set()
sns.set_style("white")
sns.set_style("ticks")
sns.set_context("poster")
import pickle
import os

import sys
myList = [[r'/home/research/cgaurav/01_fava_work/Packages/']]
myList.append(sys.path[1:])
myList = [aPath for aPart in myList for aPath in aPart]
sys.path = myList
import b2t as BT   
import binary2txt as btext

In [2]:
"""The following function takes in pos_1 (e.g. [0,1,2]) and pos_2, as well as box dims
[[xLo,xHi],[yLo,yHi],[zLo,zHi]]"""

def minimum_image_distance(pos_1,pos_2,box):
    dist=np.subtract(pos_1,pos_2)
    
    d_x=np.abs(dist[0])
    d_y=np.abs(dist[1])
    d_z=np.abs(dist[2])
    
    box_X=(box[0][1]-box[0][0])
    box_Y=(box[1][1]-box[1][0])
    box_Z=(box[2][1]-box[2][0])
    
    D_x=d_x 
    D_y=d_y 
    D_z=d_z 
    
    while (D_x>(box_X/2)):
        D_x=d_x-box_X
    while (D_y>(box_Y/2)):
        D_y=d_y-box_Y
    while (D_z>(box_Z/2)):
        D_z=d_z-box_Z
    
    distance_wa=np.sqrt(D_x*D_x+D_y*D_y+D_z*D_z)
    return(distance_wa)

In [3]:
"""
The following function takes in positions with x,y,z (e.g.np.array([[0,1,2],[0,23,19]]) and box dimensions in
[[xLo,xHi],[yLo,yHi],[zLo,zHi]]
"""
def center_of_mass(positions, box):
    N_p=len(positions) ###Find no. of atoms
    
    center_vector=[box[0][1]-box[0][0],box[1][1]-box[1][0],box[2][1]-box[2][0]] ### Vector poining to middle of the box
    ### If the systems size is -Lx,Lx,-Ly,Ly,-Lz,Lz, then center_vector=2Lx,2Ly,2Lz
    ### Translate the system by Lx, Ly, Lz so that we have all positive coordinates
    com=np.zeros(3)
    for dim in range(3):
        pos_dim=positions[:,dim]
        ##Add L to each coordinate and convert it to theta
        theta_dim=[(ele+center_vector[dim]/2)*np.pi*2/(center_vector[dim]) for ele in pos_dim]
        eta_dim=[np.cos(ele) for ele in theta_dim]
        zeta_dim=[np.sin(ele) for ele in theta_dim]
        mean_eta=np.mean(eta_dim)
        mean_zeta=np.mean(zeta_dim)
        
        mean_theta=np.arctan2(-mean_zeta,-mean_eta)+np.pi
        com[dim]=center_vector[dim]*mean_theta/2/np.pi
        com[dim]-=(center_vector[dim]/2)
        
        if (com[dim]<box[dim][0]):
            com[dim]+=center_vector[dim]
        
    return(com)


In [4]:
def any_common_elements(list1, list2):
  
  set1 = set(list1)
  flag=False
  # Check if any element in set1 is also in list2.
  for element in set1:
    if element in list2:
        flag=True

  return flag

def find_common_elements(list1, list2):
    # Convert lists to sets and find the intersection
    common_elements = set(list1).intersection(set(list2))
    return list(common_elements)


In [5]:
def find_common_elements_with_indices(list1, list2):
    # Convert lists to sets and find the intersection
    common_elements = set(list1).intersection(set(list2))
    
    # Initialize a list to store indices of common elements
    common_indices = []
    
    # Iterate through the common elements
    for elem in common_elements:
        # Find indices of elem in both lists and append to common_indices
        indices1 = [i for i, x in enumerate(list1) if x == elem]
        indices2 = [i for i, x in enumerate(list2) if x == elem]
        common_indices.append((elem, indices1, indices2))
    
    return common_indices

In [6]:
def partner_array(resid_interest,cutoff,universe):
    array_partner=[resid_interest]
    atom_list = universe.select_atoms("around {:.02f} (resid {:d}) ".format(cutoff, resid_interest))
    #lookup=range(500)
    
    for atom in atom_list:
        array_partner.append(atom.resid)
    myset=set(array_partner)
    return list(myset)

def max_clu(u,total_chains):

    partners_list=[partner_array(ele,2.5,u) for ele in range(1,total_chains+1)]
    max_length = 0
    max_list = []

    for lst in partners_list:
        if len(lst) > max_length:
            max_length = len(lst)
            max_list = lst
    
    #print(max_list)
    chain_in_cluster=max_list
    for chains in range(total_chains):
        if((any_common_elements(chain_in_cluster,partners_list[chains]))==True):
            chain_in_cluster+=(partners_list[chains])
    chain_in_cluster=set(chain_in_cluster)
    return(list(chain_in_cluster))

In [6]:
from scipy import optimize
import pickle
def fit(x, dense, dilute, shift, width):
        return 0.5 * (dense + dilute) - 0.5 * (dense - dilute) * np.tanh((2 * (x - shift)) / width)
def conc_analytic(y,x_range):

    norm_den=[np.log10(ele) for ele in y]
    #Perform the fit
    try:
        popt_dilute, pcov_dilute = optimize.curve_fit(fit, x_range,(norm_den),p0=[np.log10(norm_den[0]),(np.mean(norm_den[-50:])),10,10])
    except RuntimeError:
        popt_dilute = np.zeros(4)
        pcov_dilute = np.zeros((4, 4))
    perr_dilute = np.sqrt(np.diag(pcov_dilute))

    #Save fitting parameters 
    dense_conc = 10**popt_dilute[0]
    dilute_conc= 10**popt_dilute[1]
    interface_mid = popt_dilute[2]
    interface_width = popt_dilute[3]
    return(dilute_conc,dense_conc,interface_mid,interface_width)

In [7]:
def create_lattice_grid(x, y, z, l):
    # Calculate the number of lattice points in each dimension
    nx = int(x / l)
    ny = int(y / l)
    nz = int(z / l)

    # Create coordinate arrays for each dimension
    x_coords = np.linspace(0, x, nx, endpoint=False)
    y_coords = np.linspace(0, y, ny, endpoint=False)
    z_coords = np.linspace(0, z, nz, endpoint=False)

    # Create the meshgrid of coordinates
    xx, yy, zz = np.meshgrid(x_coords, y_coords, z_coords, indexing='ij')

    # Reshape the coordinates to obtain a flat grid
    lattice_grid = np.vstack((xx.flatten(), yy.flatten(), zz.flatten())).T

    return lattice_grid

In [8]:
import numpy as np

def calculate_center_of_mass(intensity_array):
    # Create coordinate grid for the pixel locations
    x_coords, y_coords = np.meshgrid(np.arange(intensity_array.shape[1]), np.arange(intensity_array.shape[0]))
    
    # Calculate total intensity and the weighted sum of coordinates
    total_intensity = np.sum(intensity_array)
    weighted_sum_x = np.sum(intensity_array * x_coords)
    weighted_sum_y = np.sum(intensity_array * y_coords)
    
    # Calculate center of mass coordinates
    center_of_mass_x = weighted_sum_x / total_intensity
    center_of_mass_y = weighted_sum_y / total_intensity
    
    return center_of_mass_x, center_of_mass_y




In [9]:
"""
The following function takes in positions with x,y,z (e.g.np.array([[0,1,2],[0,23,19]]) and box dimensions in
[[xLo,xHi],[yLo,yHi],[zLo,zHi]]
"""
def center_of_mass(positions, box):
    N_p=len(positions) ###Find no. of atoms
    
    center_vector=[box[0][1]-box[0][0],box[1][1]-box[1][0],box[2][1]-box[2][0]] ### Vector poining to middle of the box
    ### If the systems size is -Lx,Lx,-Ly,Ly,-Lz,Lz, then center_vector=2Lx,2Ly,2Lz
    ### Translate the system by Lx, Ly, Lz so that we have all positive coordinates
    com=np.zeros(3)
    for dim in range(3):
        pos_dim=positions[:,dim]
        ##Add L to each coordinate and convert it to theta
        theta_dim=[(ele+center_vector[dim]/2)*np.pi*2/(center_vector[dim]) for ele in pos_dim]
        eta_dim=[np.cos(ele) for ele in theta_dim]
        zeta_dim=[np.sin(ele) for ele in theta_dim]
        mean_eta=np.mean(eta_dim)
        mean_zeta=np.mean(zeta_dim)
        
        mean_theta=np.arctan2(-mean_zeta,-mean_eta)+np.pi
        com[dim]=center_vector[dim]*mean_theta/2/np.pi
        com[dim]-=(center_vector[dim]/2)
        
        if (com[dim]<box[dim][0]):
            com[dim]+=center_vector[dim]
        
    return(com)

In [10]:
def gyration_tensor(positions,box_wa):
    
    com=center_of_mass(positions,box_wa)

    positions_centered = positions - com

    gyration_tensor = np.zeros((3, 3))
    for i in range(3):
        for j in range(3):
            gyration_tensor[i, j] = np.sum(positions_centered[:, i] * positions_centered[:, j])

    gyration_tensor /= len(positions)

    return gyration_tensor

In [103]:
from MDAnalysis.analysis.density import DensityAnalysis
from MDAnalysis import transformations as trans
from MDAnalysis.analysis.density import DensityAnalysis

box_wa=[[0,64],[0,64],[0,100]]
X_INTEREST=[32]
Y_INTEREST=[32]

u=mdan.Universe('lammps.data','traj.lammpstrj',format="LAMMPSDUMP")

N_r_x_t, N_r_y_t = [],[]

for ts in u.trajectory[-1250:]:
    chain_clu=max_clu(u,700)
    res_in_cluster=('resid ')
    for kl in chain_clu:
        res_in_cluster+=(str(kl)+" ")
    
    chain_in_cluster=u.select_atoms(res_in_cluster)
        
    ### Translate the frame of reference to center of mass
    
    center_coord=center_of_mass(chain_in_cluster.positions,box_wa)
    
    translation_vector=np.array([32,32,50])-center_coord
    
    new_atom_positions=u.atoms.positions+translation_vector
    box_vectors = ts.dimensions[:3] 
    new_atom_positions=np.mod(new_atom_positions,box_vectors)
    u.atoms.positions=new_atom_positions   
    
    
    center_coord=center_of_mass(chain_in_cluster.positions,box_wa)
    
    hist_r=[[] for y_frame in range(len(Y_INTEREST))]
    i=0
    for y_center in Y_INTEREST:
        distance_list=[]
        atoms_to_select='prop y >= {:.01f} and prop y <= {:.01f}'.format(y_center-0.5,y_center+0.5)
        atom_in_y=u.select_atoms(atoms_to_select)
        
        for atom_pos in atom_in_y.positions:
            distance_list.append(minimum_image_distance(atom_pos,[32,y_center,50],box_wa))

        hist_r[i],dist_r=np.histogram(distance_list,bins=range(0,71))
        i+=1
    N_r_y_t.append(hist_r)
    
    ### N(r,x)
    hist_r=[[] for x_frame in range(len(X_INTEREST))]
    i=0
    for x_center in X_INTEREST:
        distance_list=[]
        atoms_to_select='prop x >= {:.01f} and prop x <= {:.01f}'.format(x_center-0.5,x_center+0.5)
        atom_in_x=u.select_atoms(atoms_to_select)
        
        for atom_pos in atom_in_x.positions:
            distance_list.append(minimum_image_distance(atom_pos,[x_center,32,50],box_wa))

        hist_r[i],dist_r=np.histogram(distance_list,bins=range(0,71))
        i+=1
    N_r_x_t.append(hist_r)
    

avg_N_r_y_t=np.mean(N_r_y_t,axis=0)
std_N_r_y_t=np.std(N_r_y_t,axis=0)/np.sqrt(len(N_r_y_t))

avg_N_r_x_t=np.mean(N_r_x_t,axis=0)
std_N_r_x_t=np.std(N_r_x_t,axis=0)/np.sqrt(len(N_r_x_t))


A_r=[np.pi*r*r for r in np.arange(0,50,1)]

for r in np.arange(50,50*1.414+1,1):

    A_circle=np.pi*r*r
    A_outside=8*(1/2*r*r*np.arccos(50/r)-1/2*np.sqrt(r*r-50*50)*50)
    A_r.append(A_circle-A_outside)

radial_area=[(A_r[ele+1]-A_r[ele]) for ele in range(int(50*1.414))]

for i in range(len(Y_INTEREST)):
    density_radial=[avg_N_r_y_t[i][ele]/radial_area[ele] for ele in range(70)]
    density_radial_err=[std_N_r_y_t[i][ele]/radial_area[ele] for ele in range(70)]
    
    mean_file="slices_from_COC_thick_1/dense_phase_radial_Y_{:d}.npy".format(Y_INTEREST[i])
    np.save(mean_file,density_radial)
    
    err_file="slices_from_COC_thick_1/Std_dev_dense_phase_radial_Y_{:d}.npy".format(Y_INTEREST[i])
    np.save(err_file,density_radial_err)


for i in range(len(X_INTEREST)):
    density_radial=[avg_N_r_x_t[i][ele]/radial_area[ele] for ele in range(70)]
    density_radial_err=[std_N_r_x_t[i][ele]/radial_area[ele] for ele in range(70)]
    
    mean_file="slices_from_COC_thick_1/dense_phase_radial_X_{:d}.npy".format(X_INTEREST[i])
    np.save(mean_file,density_radial)
    
    err_file="slices_from_COC_thick_1/Std_dev_dense_phase_radial_X_{:d}.npy".format(X_INTEREST[i])
    np.save(err_file,density_radial_err)

In [ ]:
### Obtain velocity of chains in dilute vs dense phase

from MDAnalysis.analysis.density import DensityAnalysis
from MDAnalysis import transformations as trans
from MDAnalysis.analysis.density import DensityAnalysis

box_wa=[[0,64],[0,64],[0,100]]


u=mdan.Universe('lammps.data','traj.lammpstrj',format="LAMMPSDUMP")



old_list_dilute,old_list_dense,old_pos_dilute,old_pos_dense=[],[],[],[]
passive_chains_condensate,total_chains_condensate=[],[]
v_dilute,v_dense=[],[]

for ts in u.trajectory[1250:]:
    chain_clu=max_clu(u,700)
    res_in_cluster=('resid ')
    for kl in chain_clu:
        res_in_cluster+=(str(kl)+" ")
    
    chain_in_cluster=u.select_atoms(res_in_cluster)
    
    dilute_phase_chains=[ele for ele in range(651) if ele not in chain_clu]
    res_in_dilute=('resid ')
    for kl in dilute_phase_chains:
        res_in_dilute+=(str(kl)+" ")
    
    chain_in_dilute=u.select_atoms(res_in_dilute)
    
    #### Find out number of active vs passive chains in the dense phase
    total_chains_condensate.append(len(chain_in_cluster)/25)
    
    active_type=[atoms.type for atoms in chain_in_cluster[0::25] if atoms.type=='2']
    passive_chains_condensate.append(len(chain_in_cluster)/25-len(active_type))
    
    ####
    
    #### Find out velocity (displacement in a timestep)
    active_beads_in_dense=[atoms.id for atoms in chain_in_cluster[0::25]]
    active_beads_in_dilute=[atoms.id for atoms in chain_in_dilute[0::25]]
    
    active_beads_pos_in_dense=chain_in_cluster[0::25].positions
    active_beads_pos_in_dilute=chain_in_dilute[0::25].positions
    
    common_dilute=find_common_elements_with_indices(old_list_dilute,active_beads_in_dilute)
    common_dense=find_common_elements_with_indices(old_list_dense,active_beads_in_dense)
    
    for i,j,k in common_dilute:
        v_dilute.append(minimum_image_distance(old_pos_dilute[j[0]],active_beads_pos_in_dilute[k[0]],box_wa))
    
    for i,j,k in common_dense:
        v_dense.append(minimum_image_distance(old_pos_dense[j[0]],active_beads_pos_in_dense[k[0]],box_wa))
        
    
    
    

np.save("v_dense.npy",v_dense)
np.save("v_dilute.npy",v_dilute)
